### Open .csv file

In [3]:
# !pip install pandas
import pandas as pd

In [20]:
df = pd.read_csv('../dataset/tweets_sentiment_covid_fix.csv')
df.head(100)

,created_at,username,text_clean,text_preprocessed,polarity_score,polarity
0,Mon Dec 04 14:38:54 +0000 2023,lillkickity,plissss lah masa w ke surabaya pas covid mulu dah,"['plissss', 'w', 'surabaya', 'pas', 'covid', '...",-1,negative
1,Sun Dec 03 22:33:34 +0000 2023,Abiiiii789,mutiara yang enak di keluarga btw bakpia tugu ...,"['mutiara', 'enak', 'keluarga', 'btw', 'bakpia...",-8,negative
2,Sun Dec 03 13:21:32 +0000 2023,zona_raya,profil doni monardo mantan kepala bnpb dan ket...,"['profil', 'doni', 'monardo', 'mantan', 'kepal...",-8,negative
3,Sun Dec 03 13:18:30 +0000 2023,e100ss,doni monardo mantan ketua satgas covid berpula...,"['doni', 'monardo', 'mantan', 'ketua', 'satgas...",-3,negative
4,Thu Nov 30 15:55:40 +0000 2023,Sudrajat110,maxim penganti go massage go life dulu saya ik...,"['maxim', 'anti', 'go', 'massage', 'go', 'life...",-4,negative
...,...,...,...,...,...,...
95,Sat Jun 17 05:15:30 +0000 2023,ANDROBET01,mantab jogetnya link mendaftar,"['mantab', 'joget', 'link', 'daftar']",-3,negative
96,Fri Jun 16 00:32:59 +0000 2023,RadioIdolaSMG,topik diskusi pagi ini indonesia akan segera ...,"['topik', 'diskusi', 'pagi', 'indonesia', 'mas...",-18,negative
97,Mon Jun 12 13:37:17 +0000 2023,fikimridho,sempronya di kos di surabaya sidangnya di ruma...,"['sempronya', 'kos', 'surabaya', 'sidang', 'ru...",5,positive
98,Mon Jun 12 09:36:11 +0000 2023,superradioid,sr surabaya mulai juni pelanggan kereta api...,"['sr', 'surabaya', 'juni', 'langgan', 'kereta'...",-17,negative


In [21]:
# 'ind' or 'en'
LANGUAGE='ind'

In [22]:
df.dtypes

created_at           object
username             object
text_clean           object
text_preprocessed    object
polarity_score        int64
polarity             object
dtype: object

In [84]:
df = df.drop('Unnamed: 4', axis=1)

In [85]:
df.dtypes

NO                     int64
NAMA_PENGGUNA         object
COMMENT               object
SENTIMEN              object
cleanComment          object
resultCaseFolding     object
resultReplaceSlang    object
resultTokenize        object
dtype: object

In [147]:
# df['text_cleaning'] = df['COMMENT'].astype(str)
df = df.astype(str)

id_komentar      object
nama_akun        object
tanggal          object
text_cleaning    object
sentimen         object
dtype: object

### Cleaning

In [23]:
import re

In [24]:
def cleanComment(comment):
    try:
        # misal ada "kawan-kawan"
        comment = re.sub(r'(?<=\w)-(?=\w)', 'STRIP', comment)
        comment = re.sub(r'http\S+|www\S+', '', comment)
        comment = re.sub(r'@\w+|[^\w\s-]|(?<!\w)-(?!\w)|\d+|(?<=\n)[IVXLCDM]+', ' ', comment)
        comment = re.sub(r'\s+', ' ', comment)
        comment = comment.replace('STRIP', '-')
        comment = comment.replace('\n', ' ')
        return comment.strip()
    except Exception as e: 
        print(f"Err: Failed to clean comments due to {str(e)}")
        return comment

In [25]:
df['cleanComment'] = df['text_clean'].apply(lambda x: cleanComment(x))
df[['text_clean', 'cleanComment']].head(100)

,text_clean,cleanComment
0,plissss lah masa w ke surabaya pas covid mulu dah,plissss lah masa w ke surabaya pas covid mulu dah
1,mutiara yang enak di keluarga btw bakpia tugu ...,mutiara yang enak di keluarga btw bakpia tugu ...
2,profil doni monardo mantan kepala bnpb dan ket...,profil doni monardo mantan kepala bnpb dan ket...
3,doni monardo mantan ketua satgas covid berpula...,doni monardo mantan ketua satgas covid berpula...
4,maxim penganti go massage go life dulu saya ik...,maxim penganti go massage go life dulu saya ik...
...,...,...
95,mantab jogetnya link mendaftar,mantab jogetnya link mendaftar
96,topik diskusi pagi ini indonesia akan segera ...,topik diskusi pagi ini indonesia akan segera m...
97,sempronya di kos di surabaya sidangnya di ruma...,sempronya di kos di surabaya sidangnya di ruma...
98,sr surabaya mulai juni pelanggan kereta api...,sr surabaya mulai juni pelanggan kereta api ka...


### Case folding

In [26]:
def caseFolding(comment):
    try:
        str(comment)
        cleanComment = comment.lower()
        return cleanComment     
    except Exception as e: 
        print(f"Err: Failed to case folding due to {str(e)}")
        return comment

In [27]:
df['resultCaseFolding'] = df['cleanComment'].apply(caseFolding)
df[['cleanComment', 'resultCaseFolding']].head(100)

,cleanComment,resultCaseFolding
0,plissss lah masa w ke surabaya pas covid mulu dah,plissss lah masa w ke surabaya pas covid mulu dah
1,mutiara yang enak di keluarga btw bakpia tugu ...,mutiara yang enak di keluarga btw bakpia tugu ...
2,profil doni monardo mantan kepala bnpb dan ket...,profil doni monardo mantan kepala bnpb dan ket...
3,doni monardo mantan ketua satgas covid berpula...,doni monardo mantan ketua satgas covid berpula...
4,maxim penganti go massage go life dulu saya ik...,maxim penganti go massage go life dulu saya ik...
...,...,...
95,mantab jogetnya link mendaftar,mantab jogetnya link mendaftar
96,topik diskusi pagi ini indonesia akan segera m...,topik diskusi pagi ini indonesia akan segera m...
97,sempronya di kos di surabaya sidangnya di ruma...,sempronya di kos di surabaya sidangnya di ruma...
98,sr surabaya mulai juni pelanggan kereta api ka...,sr surabaya mulai juni pelanggan kereta api ka...


### Slang Word

In [28]:
import os

In [40]:
def replaceSlangWords(comment, language):
    try:
        slang_dict = {}
        filename = f'../slang/slang-{language}.txt'
        if os.path.exists(filename) and os.path.getsize(filename) > 0:
            with open(filename, 'r') as f:
                for line in f:
                    slang, formal = line.strip().split(',')
                    slang_dict[slang] = formal

        words = comment.split()
        for i in range(len(words)):
            if words[i] in slang_dict:
                words[i] = slang_dict[words[i]]
        return ' '.join(words)
    except Exception as e:
        print(f"Err: Failed to replace slang words due to {str(e)}")
        return comment 

In [45]:
df['resultReplaceSlang'] = df['resultCaseFolding'].apply(lambda x: replaceSlangWords(x, LANGUAGE))
df[['resultCaseFolding', 'resultReplaceSlang']]

,resultCaseFolding,resultReplaceSlang
0,plissss lah masa w ke surabaya pas covid mulu dah,plissss lah masa w ke surabaya pas covid mulu dah
1,mutiara yang enak di keluarga btw bakpia tugu ...,mutiara yang enak di keluarga btw bakpia tugu ...
2,profil doni monardo mantan kepala bnpb dan ket...,profil doni monardo mantan kepala bnpb dan ket...
3,doni monardo mantan ketua satgas covid berpula...,doni monardo mantan ketua satgas covid berpula...
4,maxim penganti go massage go life dulu saya ik...,maxim penganti go massage go life dulu saya ik...
...,...,...
1300,heheh ketemu setaon skali belum tentu sejak pa...,heheh ketemu setahun sekali belum tentu sejak ...
1301,kamu kuliah dimana nder aku sebelum pandemi di...,kamu kuliah dimana nder aku sebelum pandemi di...
1302,terima kasih kepada para wisatawan yang sudah ...,terima kasih kepada para wisatawan yang sudah ...
1303,kanwil kemenkumham jatim wisnu nugroho dewanto...,kanwil kemenkumham jatim wisnu nugroho dewanto...


### Tokenizing

In [46]:
def tokenize(comment):
    try:
        words = comment.split(' ')
        words = list(filter(None, words)) 

        return words
    except Exception as e:
        print("Err: Failed to tokenize due to", str(e))
        return comment

In [47]:
df['resultTokenize'] = df['resultReplaceSlang'].apply(tokenize)
df[['resultReplaceSlang', 'resultTokenize']].head(20)

,resultReplaceSlang,resultTokenize
0,plissss lah masa w ke surabaya pas covid mulu dah,"[plissss, lah, masa, w, ke, surabaya, pas, cov..."
1,mutiara yang enak di keluarga btw bakpia tugu ...,"[mutiara, yang, enak, di, keluarga, btw, bakpi..."
2,profil doni monardo mantan kepala bnpb dan ket...,"[profil, doni, monardo, mantan, kepala, bnpb, ..."
3,doni monardo mantan ketua satgas covid berpula...,"[doni, monardo, mantan, ketua, satgas, covid, ..."
4,maxim penganti go massage go life dulu saya ik...,"[maxim, penganti, go, massage, go, life, dulu,..."
5,bi sebut qris selamatkan indonesia dari krisis...,"[bi, sebut, qris, selamatkan, indonesia, dari,..."
6,buanyak sekali usia an yang pengangguran setel...,"[buanyak, sekali, usia, an, yang, pengangguran..."
7,naufal tapi hasilnya bagus dan konsisten kan w...,"[naufal, tapi, hasilnya, bagus, dan, konsisten..."
8,tweet kota solo adalah kota dgn pertumbuhan te...,"[tweet, kota, solo, adalah, kota, dgn, pertumb..."
9,rail clinic kembali hadir untuk masyarakat set...,"[rail, clinic, kembali, hadir, untuk, masyarak..."


### Stop Word Removal

In [48]:
# !pip install nltk

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Dell
[nltk_data]     7300\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [49]:
def stopwordRemoval(comments, language):
    language_mapping = {
        'en': 'english', 
        'ind': 'indonesian'
    }

    nltk_language = language_mapping.get(language, 'english') 
    stopWordRemoved = []
    
    try:
        stopList = stopwords.words(nltk_language)
        filename = f'../stopword/stopword-{language}.txt' 
        if os.path.exists(filename) and os.path.getsize(filename) > 0:
            with open(filename, 'r') as f:
                stopList.extend(f.read().split('\n')) 
        for word in comments:
            if word not in stopList:
                stopWordRemoved.append(word)

        return stopWordRemoved  
    except Exception as e:  
        print(f"Err: Failed to remove stopwords due to {str(e)}")
        return comments 

In [50]:
df['resultStopword'] = df['resultTokenize'].apply(lambda x: stopwordRemoval(x, LANGUAGE))
df[['resultTokenize', 'resultStopword']].head(20)

,resultTokenize,resultStopword
0,"[plissss, lah, masa, w, ke, surabaya, pas, cov...","[plissss, surabaya, pas, covid, mulu, dah]"
1,"[mutiara, yang, enak, di, keluarga, btw, bakpi...","[mutiara, enak, keluarga, btw, bakpia, tugu, l..."
2,"[profil, doni, monardo, mantan, kepala, bnpb, ...","[profil, doni, monardo, mantan, kepala, bnpb, ..."
3,"[doni, monardo, mantan, ketua, satgas, covid, ...","[doni, monardo, mantan, ketua, satgas, covid, ..."
4,"[maxim, penganti, go, massage, go, life, dulu,...","[maxim, penganti, go, massage, go, life, gomas..."
5,"[bi, sebut, qris, selamatkan, indonesia, dari,...","[bi, qris, selamatkan, indonesia, krisis, pand..."
6,"[buanyak, sekali, usia, an, yang, pengangguran...","[buanyak, usia, an, pengangguran, covidf, sura..."
7,"[naufal, tapi, hasilnya, bagus, dan, konsisten...","[naufal, hasilnya, bagus, konsisten, tergebrak..."
8,"[tweet, kota, solo, adalah, kota, dgn, pertumb...","[tweet, kota, solo, kota, dgn, pertumbuhan, te..."
9,"[rail, clinic, kembali, hadir, untuk, masyarak...","[rail, clinic, hadir, masyarakat, vakum, akiba..."


### Hapus data kosong ([])

In [51]:
df = df[df['resultStopword'].apply(lambda x: len(x) > 0)]
df[['resultStopword']]

,resultStopword
0,"[plissss, surabaya, pas, covid, mulu, dah]"
1,"[mutiara, enak, keluarga, btw, bakpia, tugu, l..."
2,"[profil, doni, monardo, mantan, kepala, bnpb, ..."
3,"[doni, monardo, mantan, ketua, satgas, covid, ..."
4,"[maxim, penganti, go, massage, go, life, gomas..."
...,...
1300,"[heheh, ketemu, setahun, pandemi, sdh, taon, g..."
1301,"[kuliah, dimana, nder, pandemi, surabaya, jata..."
1302,"[terima, kasih, wisatawan, berkunjung, gili, t..."
1303,"[kanwil, kemenkumham, jatim, wisnu, nugroho, d..."


### Stemming

In [52]:
# !pip install Sastrawi

from nltk.stem import PorterStemmer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [53]:
def stemmed_wrapper(term, language):
    try:
        if language == 'ind':
            factory = StemmerFactory()
            stemmer = factory.create_stemmer()
            return stemmer.stem(term)
        else: 
            stemmer = PorterStemmer()
            return stemmer.stem(term)
    except Exception as e:
        print(f"Err: Failed to stem term '{term}' due to {str(e)}")
        return term 

def stemming(document, language):
    return [stemmed_wrapper(term, language) for term in document]

In [54]:
df['resultStemming'] = df['resultStopword'].apply(lambda x: stemming(x, LANGUAGE))

In [55]:
df[['resultStopword', 'resultStemming']]

,resultStopword,resultStemming
0,"[plissss, surabaya, pas, covid, mulu, dah]","[plissss, surabaya, pas, covid, mulu, dah]"
1,"[mutiara, enak, keluarga, btw, bakpia, tugu, l...","[mutiara, enak, keluarga, btw, bakpia, tugu, l..."
2,"[profil, doni, monardo, mantan, kepala, bnpb, ...","[profil, doni, monardo, mantan, kepala, bnpb, ..."
3,"[doni, monardo, mantan, ketua, satgas, covid, ...","[doni, monardo, mantan, ketua, satgas, covid, ..."
4,"[maxim, penganti, go, massage, go, life, gomas...","[maxim, anti, go, massage, go, life, gomassage..."
...,...,...
1300,"[heheh, ketemu, setahun, pandemi, sdh, taon, g...","[heheh, ketemu, tahun, pandemi, sdh, taon, gak..."
1301,"[kuliah, dimana, nder, pandemi, surabaya, jata...","[kuliah, mana, nder, pandemi, surabaya, jatah,..."
1302,"[terima, kasih, wisatawan, berkunjung, gili, t...","[terima, kasih, wisatawan, kunjung, gili, traw..."
1303,"[kanwil, kemenkumham, jatim, wisnu, nugroho, d...","[kanwil, kemenkumham, jatim, wisnu, nugroho, d..."


### Export clean data

In [55]:
df.to_csv(
    './output/CLEAN-dataset_tweet_sentimen_tayangan_tv.csv',
    columns=[
        'Id',
        'Sentiment',
        'Acara TV',
        'Jumlah Retweet',
        'Text Tweet',
        'cleanComment',
        'resultCaseFolding',
        'resultReplaceSlang',
        'resultTokenize',
        'resultStopword',
        'resultStemming',
    ],
    index=False
)

In [97]:
df.to_csv(
    './output/CLEAN-dataset_tweet_sentiment_cellular_service_provider.csv',
    columns=df.columns,
    index=False
)

In [130]:
df.to_csv(
    './output/CLEAN-dataset_tweet_sentiment_opini_film.csv',
    columns=df.columns,
    index=False
)

In [38]:
df.to_csv(
    '../output/CLEAN-dataset_mobil_listrik.csv',
    columns=df.columns,
    index=False
)

In [68]:
df.to_csv(
    './output/data-clean-ind-new.csv', 
    columns=[
        'Comment', 
        'cleanComment',
        'resultCaseFolding',
        'resultReplaceSlang',
        'resultTokenize',
        'resultStopword',
        'resultStemming',
    ],
    index=False
)

In [67]:
df.to_excel(
    './output/data-clean-ind-new.xlsx', 
    columns=[
        'Comment', 
        'cleanComment',
        'resultCaseFolding',
        'resultReplaceSlang',
        'resultTokenize',
        'resultStopword',
        'resultStemming',
    ],
    index=False
)